In [9]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Union
from calendar import c
from typing import Literal


In [10]:
def accuracy(y_hat: pd.Series, y: pd.Series):
    assert y_hat.size == y.size

    correct_predictions = sum(a == p for a, p in zip(y, y_hat))
    total_predictions = len(y)
    
    accuracy = correct_predictions / total_predictions
    return accuracy
    

def precision(y_hat: pd.Series, y: pd.Series, cls: Union[int, str]):
    true_positives=y_hat[y == cls].value_counts().get(cls,0)
    total_prediction=len([y_hat == cls])

    # print(y_hat)
    # print("true_postivi" , true_positives)

    if total_prediction == 0:
        return 0.0  # Avoid division by zero

    precision = true_positives / total_prediction
    return precision


def recall(y_hat: pd.Series, y: pd.Series, cls: Union[int, str]):
    true_positives=y_hat[y == cls].value_counts().get(cls,0)
    total_actual=len([y == cls])

    if total_actual == 0:
        return 0.0
    recall = true_positives / total_actual
    return recall


def rmse(y_hat: pd.Series, y: pd.Series):
    assert ValueError("Input lists must have the same length.")

    squared_errors = pd.sub(y, y_hat) ** 2 
    mean_squared_error = squared_errors.mean()
    rmse = np.sqrt(mean_squared_error)
    return rmse


def mae(y_hat: pd.Series, y: pd.Series):
    assert ValueError("Input lists must have the same length.")

    squared_errors = pd.sub(y, y_hat) ** 2 
    return squared_errors.mean()


In [11]:
def check_ifreal(y: pd.Series):
    if y.dtype.name == "category":
        return False
    return True

#calculated on target variable
def entropy(Y: pd.Series):
    unique_class, class_counts = np.unique(Y, return_counts=True)
    probability = class_counts/len(Y)
    return -np.sum(probability * np.log2(probability))

def gini_index(y):
    unique_class, class_counts = np.unique(y, return_counts=True)
    probability = class_counts/len(y)
    return 1-np.sum(probability ** 2)

def MSE():
    pass

def information_gain(attr: pd.DataFrame, Y: pd.Series, feature_idx):
    # conataining unique value of particular feature
    unique_classs = np.unique(attr[feature_idx])
    node_entropy=0
    total_child_entropy = 0

    #entropy on whole dataset target label
    node_entropy = entropy(Y)

    for value in unique_classs:
        # taking sub part of target which has same value for the feature
        child_target = Y[attr[feature_idx] == value]
        total_child_entropy += len(child_target) / \
            len(Y) * entropy(child_target)
  
    return node_entropy-total_child_entropy


def opt_split_attribute(X: pd.DataFrame, y: pd.Series, criterion="information_gain"):
    """
    Function to find the optimal attribute to split about.
    If needed you can split this function into 2, one for discrete and one for real valued features.
    You can also change the parameters of this function according to your implementation.

    features: pd.Series is a list of all the attributes we have to split upon

    return: attribute to split upon
    """

    # According to wheather the features are real or discrete valued and the criterion, find the attribute
    #  from the features series with the maximum information gain
    #  (entropy or varinace based on the type of output) or minimum gini index (discrete output).

    best_feature=0

    if criterion =="information_gain":
        max_info_gain=0
        for column_name in X.columns:
            new_info_gain=information_gain(X,y,column_name)
            if new_info_gain > max_info_gain:
                max_info_gain=new_info_gain
                best_feature=column_name
    else :
        gini_value=float('inf')
        for column_name in X.columns:
            new_gini=gini_index(X[column_name])
            if new_gini < gini_value:
                gini_value=new_gini
                best_feature=column_name
    
    return best_feature


def split_data(X: pd.DataFrame, y: pd.Series, attribute, value):
    """
    Funtion to split the data according to an attribute.
    If needed you can split this function into 2, one for discrete and one for real valued features.
    You can also change the parameters of this function according to your implementation.

    attribute: attribute/feature to split upon
    value: value of that attribute to split upon

    return: splitted data(Input and output)
    """


    # Split the data based on a particular value of a particular attribute. 
    # You may use masking as a tool to split the data.

    

In [12]:
class Node:
    def __init__(self, data=None, feature_value=None,target=None,child=None,result=None):
        self.data = data  # data corresponding to the node [matrix]
        self.target = target  # y data
        self.children = child  # child names & objects
        self.feature = feature_value  # value at node
        self.result = result

    def plot(self, level=0):
        indent = "  " * level
        print(f"{indent}|- {self.feature}: {self.result}")
        if self.children is not None:
            for child in self.children:
                self.children[child].plot(level + 1)

class DecisionTree:
    # criterion won't be used for regression
    criterion: Literal["information_gain", "gini_index"]
    max_depth: int  # The maximum depth the tree can grow to
    root = None

    def __init__(self, criterion, max_depth=5):
        self.criterion = criterion
        self.max_depth = max_depth

    # def fit_DI_DO(self, data_frame: pd.DataFrame, target: pd.Series, depth=0,criterion="information_gain"):
    #     return self.id3(data_frame,target,depth)

    def id3(self,data_frame: pd.DataFrame, target: pd.Series, depth=0,criterion="information_gain"):
            # If all instances have the same target value, create a leaf node
            if len(np.unique(target)) == 1:
                majority_class = target.mode()[0]
                return Node(result=majority_class)

            # If there are no more features to split on, create a leaf node with the majority class
            if len(list(data_frame.columns)) == 0:
                return Node(result=majority_class)

            # If maximum depth is reached, create a leaf node with the majority class
            if self.max_depth is not None and depth == self.max_depth:
                majority_class = target.mode()[0]
                return Node(result=majority_class)

            # Choose the best feature to split on based on information gain considering only single level in tree
            best_attribute=opt_split_attribute(data_frame,target,criterion=criterion)
            root=Node(data=data_frame,target=target,feature_value=best_attribute,child={})

            for value in np.unique(data_frame[best_attribute]):
                subset_idx=data_frame[best_attribute] == value
                subset_data = data_frame[subset_idx]
                subset_target = target[subset_idx]
                if len(subset_target) > 0 :
                    root.children[value]=self.id3(subset_data, subset_target)
                
            return root

    def predict_multiway(self,tree, sample):
        # Reached a leaf node which contains the result value, return the result
        if tree.result is not None:
            return tree.result

        # Not a leaf node, continue traversing the tree
        feature_value = sample[tree.feature]

        if feature_value in tree.children:
            child_node = tree.children[feature_value]
            return self.predict_multiway(child_node, sample)
        else:
            # Handle missing values or unknown values as needed
            return None

    def fit(self, X: pd.DataFrame, y: pd.Series):
        return self.id3(X,y,depth=0)     

    def predict(self, tree,X):
        return self.predict_multiway(tree,X)

    def plot(self):
        """
        Function to plot the tree

        Output Example:
        ?(X1 > 4)
            Y: ?(X2 > 7)
                Y: Class A
                N: Class B
            N: Class C
        Where Y => Yes and N => No
        """
        if self.root is not None:
            self.root.plot()

In [13]:
np.random.seed(42)
N = 30
P = 5
X = pd.DataFrame({i: pd.Series(np.random.randint(P, size=N), dtype="category") for i in range(5)})
y = pd.Series(np.random.randint(P, size=N), dtype="category")


for criteria in ["information_gain", "gini_index"]:
    tree = DecisionTree(criterion=criteria)  # Split based on Inf. Gain
    root=tree.fit(X, y)
    y_hat=[]
    for idx,x in X.iterrows():
        y_hat.append(tree.predict(root,x))
    
    # print(y_hat)
    y_hat_series=pd.Series(y_hat)
    # print(y_hat_series)

    # plt.plot(range(30),y,marker='o')
    # plt.plot(range(30),y_hat_series)
    # plt.show()

    root.plot()
    print("Criteria :", criteria)
    print("Accuracy: ", accuracy(y_hat_series, y))
    for cls in y.unique():
        print("Precision: ", precision(y_hat_series, y, cls))
        print("Recall: ", recall(y_hat_series, y, cls))

|- 3: None
  |- 0: None
    |- None: 3
    |- None: 0
    |- None: 0
    |- 1: None
      |- None: 2
      |- None: 4
    |- 1: None
      |- None: 2
      |- None: 2
      |- None: 0
  |- 1: None
    |- None: 0
    |- None: 0
    |- None: 2
    |- None: 4
  |- 0: None
    |- 1: None
      |- None: 0
      |- None: 3
    |- None: 0
    |- None: 0
    |- 1: None
      |- None: 3
      |- None: 0
      |- None: 3
    |- None: 0
  |- 2: None
    |- None: 2
    |- None: 2
    |- None: 4
  |- 1: None
    |- None: 1
    |- None: 3
    |- None: 1
Criteria : information_gain
Accuracy:  1.0
Precision:  13.0
Recall:  13.0
Precision:  6.0
Recall:  6.0
Precision:  5.0
Recall:  5.0
Precision:  3.0
Recall:  3.0
Precision:  3.0
Recall:  3.0
|- 3: None
  |- 0: None
    |- None: 3
    |- None: 0
    |- None: 0
    |- 1: None
      |- None: 2
      |- None: 4
    |- 1: None
      |- None: 2
      |- None: 2
      |- None: 0
  |- 1: None
    |- None: 0
    |- None: 0
    |- None: 2
    |- None: 4
  |- 0:

In [14]:
# tr=DecisionTree(criterion="information_gain")
# root=tr.fit(X,y)
# y_hat=[]
# for idx,x in X.iterrows():
#     y_hat.append(tr.predict_multiway(root,x))
# print(y_hat)
# y_hat_series=pd.Series(y_hat)

In [15]:
# plt.plot(range(30),y,y_hat,marker='o')